Importing necessary libraries

In [ ]:
!pip install kaggle

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.regularizers import l2

Download dataset

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d alik05/forest-fire-dataset -p /content/dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/alik05/forest-fire-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 98% 139M/142M [00:01<00:00, 103MB/s] 
100% 142M/142M [00:01<00:00, 82.9MB/s]


Data Preparation

In [ ]:
fire_images_path = '/content/dataset/Forest Fire Dataset/Training/fire'
non_fire_images_path = '/content/dataset/Forest Fire Dataset/Training/nofire'

In [ ]:
def load_data(data_dir, target_size=(250, 250)):
    # Initialize empty lists to store images and labels
    data = []
    labels = []

    categories = os.listdir(data_dir)

    for category in categories:
        # Path to the category directory
        category_dir = os.path.join(data_dir, category)
        # Get the label (0 for 'nofire' and 1 for 'fire')
        label = 1 if category == 'fire' else 0


        for img_file in os.listdir(category_dir):
            # Read the image
            img_path = os.path.join(category_dir, img_file)
            img = cv2.imread(img_path)
            # Resize the image to the target size
            img = cv2.resize(img, target_size)
            # Normalize the image pixels to be in the range [0, 1]
            img = img.astype('float32') / 255.0
            # Append the image and label to the data lists
            data.append(img)
            labels.append(label)

    return np.array(data), np.array(labels)

In [ ]:
train_data, train_labels = load_data('/content/dataset/Forest Fire Dataset/Training')
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)

Train data shape: (1216, 250, 250, 3)
Validation data shape: (304, 250, 250, 3)


In [ ]:
num_classes = 2
train_labels = to_categorical(train_labels, num_classes)
val_labels = to_categorical(val_labels, num_classes)

In [ ]:
batch_size = 32
epochs = 25
input_shape = (250, 250, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3),strides=(2, 2), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3),strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add dropout layer for regularization
model.add(Dropout(0.25))

# Flatten the output
model.add(Flatten())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 248, 248, 32)      896       
                                                                 
 batch_normalization_5 (Bat  (None, 248, 248, 32)      128       
 chNormalization)                                                
                                                                 
 conv2d_7 (Conv2D)           (None, 123, 123, 64)      18496     
                                                                 
 batch_normalization_6 (Bat  (None, 123, 123, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 61, 61, 64)        0         
 g2D)                                                            
                                                      

In [ ]:
history = model.fit(
    train_data, train_labels,
    epochs=25,
    batch_size=32,
    validation_data=(val_data, val_labels)
)

Epoch 1/25
38/38 [==============================] - 242s 6s/step - loss: 1.3559 - accuracy: 0.8923 - val_loss: 0.9366 - val_accuracy: 0.8125
Epoch 2/25
38/38 [==============================] - 247s 7s/step - loss: 1.0016 - accuracy: 0.9235 - val_loss: 0.9032 - val_accuracy: 0.7993
Epoch 3/25
38/38 [==============================] - 243s 6s/step - loss: 0.7624 - accuracy: 0.9424 - val_loss: 0.9982 - val_accuracy: 0.7171
Epoch 4/25
38/38 [==============================] - 247s 7s/step - loss: 0.7608 - accuracy: 0.9359 - val_loss: 0.7955 - val_accuracy: 0.8322
Epoch 5/25
38/38 [==============================] - 246s 6s/step - loss: 1.0322 - accuracy: 0.9400 - val_loss: 1.2217 - val_accuracy: 0.6875
Epoch 6/25
38/38 [==============================] - 254s 7s/step - loss: 0.7011 - accuracy: 0.9482 - val_loss: 2.5980 - val_accuracy: 0.5230
Epoch 7/25
38/38 [==============================] - 264s 7s/step - loss: 0.6145 - accuracy: 0.9572 - val_loss: 2.8263 - val_accuracy: 0.5526
Epoch 8/25
38

In [ ]:
# Evaluate the model on validation data
score = model.evaluate(val_data, val_labels, verbose=1)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Validation loss: 0.2835243344306946
Validation accuracy: 0.9539473652839661


## Test the model

Prepare Test data

In [ ]:
def load_test_data(data_dir, target_size=(250, 250)):
    # Initialize empty lists to store images and labels
    data = []
    labels = []

    for img_file in os.listdir(data_dir):
        img_path = os.path.join(data_dir, img_file)
        img = cv2.imread(img_path)
        # Resize the image to the target size
        img = cv2.resize(img, target_size)
        # Normalize the image pixels to be in the range [0, 1]
        img = img.astype('float32') / 255.0
        # Append the image to the data list
        data.append(img)

        # Get the label from the filename (0 for 'nofire' and 1 for 'fire')
        label = 1 if img_file.startswith('fire') else 0
        labels.append(label)

    return np.array(data), np.array(labels)

test_data, test_labels = load_test_data('/content/dataset/Forest Fire Dataset/Testing')
test_labels = to_categorical(test_labels, num_classes=2)

Evaluate

In [ ]:
# Evaluate the model on the test data
score = model.evaluate(test_data, test_labels, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

12/12 [==============================] - 22s 2s/step - loss: 0.3123 - accuracy: 0.9553
Test loss: 0.31232190132141113
Test accuracy: 0.9552631378173828
